# Docker Basics

Docker is a platform that allows you to automate the deployment of applications inside lightweight, portable containers. This tutorial will guide you through the basics of Docker: installation, pulling and running containers, building Docker images, and basic container management.

Before we begin, let's check if Docker is installed and running on your machine.

## Checking Docker Installation

In [9]:
!docker --version

Docker version 29.1.3, build f52814d


If Docker is installed, this will display the installed version. If not, please install Docker before proceeding. To avoid complex setup, it is recommended to install [Docker Desktop](https://docs.docker.com/desktop/).

## Pulling Your First Docker Image
Docker containers are built from Docker images. In this section, we'll pull a pre-built image from [Docker Hub](https://hub.docker.com/).

We'll start by pulling the official `hello-world` image.

In [10]:
!docker pull hello-world

Using default tag: latest
latest: Pulling from library/hello-world
Digest: sha256:ef54e839ef541993b4e87f25e752f7cf4238fa55f017957c2eb44077083d7a6a
Status: Image is up to date for hello-world:latest
docker.io/library/hello-world:latest


## Running a Docker Container
Now that we have the `hello-world` image, let's run it as a container.

In [11]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



When you run this, Docker will start a container using the `hello-world` image. It will print a message and then exit. The message demonstrates that your Docker installation is working properly.

## Listing Running and Stopped Containers
Docker keeps a history of the containers you've run. You can list them using the following command:

In [12]:
!docker ps -a

CONTAINER ID   IMAGE         COMMAND    CREATED          STATUS                      PORTS     NAMES
3f42d136f14e   hello-world   "/hello"   16 seconds ago   Exited (0) 14 seconds ago             hungry_clarke
7d10e4571e66   hello-world   "/hello"   50 seconds ago   Exited (0) 48 seconds ago             clever_tesla


This command will display all containers, including stopped ones, along with their ID, name, status, and other details.

## Removing a Docker Container
To clean up, let's remove the `hello-world` container we just created.

First, find the container ID or name using `docker ps -a`, and then run, replace the ID in the next command and run:

In [15]:
!docker rm 3f42d136f14e
!docker rm 7d10e4571e66

Error response from daemon: No such container: 3f42d136f14e


7d10e4571e66


## Using Shared Volumes in Docker

Shared volumes allow you to share files between your host machine and the Docker container. This is useful for editing files on your local machine while having those changes immediately reflected in the container.

We'll create a shared volume between a folder on your host machine and the container running a simple Python script.

###
First, let's create a directory on your host machine where we can store a Python script.

In [16]:
# Create directory for the shared volume in python
import os
os.makedirs("shared_volume", exist_ok=True)

### Create a Python Script on Your Host

In [17]:
%%writefile ./shared_volume/hello_from_volume.py
print("Hello from the shared volume!")

Overwriting ./shared_volume/hello_from_volume.py


## Building a Docker Image
Now, let's move on to creating our own Docker image. We'll create a simple image that runs a Python script from the shared volume.

First, create a `Dockerfile`. This file contains instructions for building a Docker image. Here's an example:

In [18]:
%%writefile Dockerfile
# Use Python base image
FROM python:3.8-slim

# Set the working directory inside the container
WORKDIR /app

# Command to run a script from the volume (this script will be provided by the volume)
CMD ["python", "/app/hello_from_volume.py"]

Overwriting Dockerfile


Now, build the Docker image using the `docker build` command. The `-t` flag allows you to tag the image with a name:

In [19]:
!docker build -t my-python-app .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 287B 0.0s done
#1 DONE 0.1s

#2 [internal] load metadata for docker.io/library/python:3.8-slim
#2 ...

#3 [auth] library/python:pull token for registry-1.docker.io
#3 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.8-slim
#2 DONE 2.8s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [1/2] FROM docker.io/library/python:3.8-slim@sha256:1d52838af602b4b5a831beb13a0e4d073280665ea7be7f69ce2382f29c5a613f
#5 resolve docker.io/library/python:3.8-slim@sha256:1d52838af602b4b5a831beb13a0e4d073280665ea7be7f69ce2382f29c5a613f 0.0s done
#5 sha256:302e3ee498053a7b5332ac79e8efebec16e900289fc1ecd1c754ce8fa047fcab 0B / 29.13MB 0.2s
#5 sha256:030d7bdc20a63e3d22192b292d006a69fa3333949f536d62865d1bd0506685cc 0B / 3.51MB 0.2s
#5 sha256:3971691a363796c39467aae4cdce6ef773273fe6bfc67154d01e1b589befb912 0B / 248B 0.

## Running the Custom Docker Image

We will now run the container, mounting the local directory (`./shared_volume`) into the container at the `/app` directory. This way, the Python script on your local machine will be accessible from inside the container.

Explanation:
- **`-v "{shared_vol_dir}:/app"`**: This option mounts the host directory (`{shared_vol_dir}`) to the container’s `/app` directory.
- **`--rm`**: Automatically removes the container once it exits.
- **`my-python-app`**: This is the image we built earlier.

In [20]:
shared_vol_dir = os.path.abspath("./shared_volume")
shared_vol_dir

'c:\\Users\\dagma\\Documents\\GitHub\\IncubatorDTCourse\\0-Pre-requisites\\shared_volume'

In [21]:
!docker run --rm -v "{shared_vol_dir}:/app" my-python-app

Hello from the shared volume!


You should see the output: `Hello from the shared volume!`.

## Editing the Script

Let’s modify the Python script from your local machine and re-run the container to see how changes are reflected:

In [22]:
%%writefile ./shared_volume/hello_from_volume.py
print("Hello again from the updated shared volume!")

Overwriting ./shared_volume/hello_from_volume.py


Run the container again:

In [23]:
!docker run --rm -v "{shared_vol_dir}:/app" my-python-app

Hello again from the updated shared volume!


The container will now output: `Hello again from the updated shared volume!`.


## Clean Up

To remove the image you created:

In [24]:
!docker rmi my-python-app

Untagged: my-python-app:latest
Deleted: sha256:d62931485dc17b09190cd4b5f50556f09636e29e20e3aa17ef3c5111e1234dec


In [25]:
# Remove all stopped containers
!docker container prune -f

Total reclaimed space: 0B
